In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install multipledispatch

# TOP DOWN APPROACH

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import random
from multipledispatch import dispatch

import warnings
warnings.simplefilter("ignore")

## data()

In [ ]:
def data(dt):
  print("We currently have this dataset.\n\n")
  dt = dt.loc[:,['Names', 'Goals_per_match', 'Assists_per_match', 'Position']]
  print(dt.head())

  return dt

## menu()

In [ ]:
def menu():
  flag = -1

  while flag<0:
    print("\nThese are the algorithms:\n1. K-Means Clustering\n2. KNN Classification\n\nEnter your choice:")
    choice = int(input())
    if choice==1 or choice==2:
      flag=0
    else:
      print("Wrong Choice\n\nEnter your choice again.")

  algos = {1: "K-Means Clustering", 2: "K-NN Classification"}

  print("\nYou have chosen: ", end='')
  print(algos[choice])

  k = int(input("\nEnter the value of k: "))

  print("\nDistances that can be used:\n1. Manhattan Distance\n2. Euclidean Distance\n")
  d = int(input("Choose the type of distance you want to use: "))

  return choice, k, d 

## Distances

## Manhattan distance

In [ ]:
@dispatch(np.ndarray, int, int)
def distance(X, n, r):
  #print("\nCalculating Manhattan Distance\n")
  a, b = [], []

  for i in range(n):
    #       target    -every value
    a += [abs(X[r][0] - X[i][0])]
    b += [abs(X[r][1] - X[i][1])]

  sum = []
  for i in range(n):
    sum+=[a[i]+b[i]]

  return sum

## Euclidean distance

In [ ]:
@dispatch(np.ndarray, int, int, int)
def distance(X, n, r, z):
  #print("\nCalculating Euclidean Distance\n")
  a, b = [], []

  for i in range(n):
    #       target    -every value
    a += [abs(X[r][0] - X[i][0])]
    b += [abs(X[r][1] - X[i][1])]

  sum = []
  for i in range(n):
    sum += [ math.sqrt(a[i]**2 + b[i]**2) ]

  return sum

## kmeans

In [ ]:
def kmeans(dt, k, d):
  print("K-MEANS")
  counter = 0
  Dict = {}
  
  X = dt.iloc[:, [1, 2]].values
  y = dt.iloc[:, [2]].values

  n = len(y)
  pca = []
  for i in range(n):
    pca+=[ (X[i][0]+X[i][1])/2 ]

  variance = []
  for w in range(10):  # for 10 random set of centroids  
    flag1, flag2 = 0, 0
    c = [0]*k         # centroid indexes
    centroid = [0]*k  # actual centroid

    # centroid index position
    for i in range(k):
        c[i] = random.randint(0, n-1)
        if c[i] in c:
          c[i] = random.randint(0, n-1)
    
    for i in range(k):
      centroid[i] = pca[c[i]] 

    

    while(flag1 == 0):
      counter+=1
      if counter>100:
        break
    #    flag1 = 1
      cent_new = [0]*k
      dist = [0]*k

      grp = []
      group = []
      
      for i in range(k): # for each centroid
        grp+=[[]]
        group+=[[]]
        
        if d==1:
          dist[i] = distance(X, n, c[i])
        elif d==2:
          dist[i] = distance(X, n, c[i], d)

      for u in range(n):    # row wise iteration
        #dist[u]
        lst=[0]*k           

        for v in range(k):  # for centroid wise iteration

          lst[v] = dist[v][u] 
        

        mn = min(lst)           

        ind = lst.index(mn)     

        grp[ind] += [ u ]  

        group[ind] += [ pca[u] ] # k-clusters are made

      # the clusters are getting new centroids
      for i in range(k):
        if len(group[i])>0:
          cent_new[i] = round(sum(group[i])/len(group[i]),4)

      # checking if the cluster centroids are changing 345, 345
      flag2 = 0
      for i in range(k):
        if centroid[i] != cent_new[i]:
          flag2=1
      flag1 = 0
      if flag2==0: # this means, the centroids (old and new) are same
        flag1 = 1
      else: 
        centroid = cent_new # will use these new centroids to make new clusters
        continue

      var = []
      
      for i in range(k):
        var += [[]]

        var[i] = np.var(group[i])

      Dict[sum(var)] = c 

      variance+=[sum(var)]

  spl_c = Dict[min(Dict.keys())]   #special centroids
  print("\nCentroids: ", spl_c)
  centroid = [0]*k

  for i in range(k):
    centroid[i] = pca[spl_c[i]]

  dist = [0]*k

  grp = []
  group = []
  
  for i in range(k): # for each centroid
    grp+=[[]]
    group+=[[]]
    
    if d==1:
      dist[i] = distance(X, n, c[i])
    elif d==2:
      dist[i] = distance(X, n, c[i], d)

  for u in range(n):    # row wise iteration
    #dist[u]
    lst=[0]*k           

    for v in range(k):  # for centroid wise iteration

      lst[v] = dist[v][u] 
    

    mn = min(lst)           

    ind = lst.index(mn)     

    grp[ind] += [ u ]  

    group[ind] += [ pca[u] ] # k-clusters are made

  for i in range(k):
    print("\n\nCluster ", i , ":\n")
    print(grp[i])
    print(dt.iloc[grp[i], [0]])

  


## knn

In [ ]:
def knn(dt, k, d):
  print("K-NN")

  X = dt.iloc[:, [2,1]].values
  y = dt.iloc[:, [-1]].values

  n = len(y)
  
  prediction = []
  actual = [] 

  for iteration in range(10):
    r = random.randint(0, n-1)
    actual.append(y[r,0])

    if(d==1):
      dist = distance(X, n, r)
    elif d==2:
      dist = distance(X, n, r, d)

    pos = y[:,0]

    df = pd.DataFrame({'sum': dist,'pos': pos})

    df = df.sort_values(by = 'sum')

    p = list(df.head(k).pos)

    class_count = dict()
    for i in p:
      class_count[i] = 0

    for i in p:
      class_count[i] += 1

    class_count_list = class_count.values()

    max_of_class_count = max(class_count_list)

    for i in class_count:
      if class_count[i]==max_of_class_count:
        prediction.append(i)

  cm = [ [0]*3 ]*3
  cm=np.array(cm)
  
  print("\nActual class values:\n", actual)
  print("\nPredicted class value:\n", prediction)
  for i in range(len(actual)):
    cm[prediction[i]][actual[i]]+=1
  print("\nConfusion Matrix:\n", cm)  

  print("\nAccuracy: ", (np.trace(cm)/np.sum(cm))*100,"%")

## algos()

In [ ]:
def algos(dt):
  choice, k, d = menu()

  if choice==1:
    kmeans(dt, k, d)
  elif choice==2:
    knn(dt, k, d)

## Driver Function

In [ ]:
# the driver function, the one which will control the whole project

def driver():
  filename = '/content/drive/MyDrive/Minor_2/fantasy_fooball.csv'
  dt = pd.read_csv(filename)

  dt = data(dt)

  algos(dt)

### DRIVE

In [ ]:
driver()

We currently have this dataset.


              Names  Goals_per_match  Assists_per_match  Position
0              Pepe         0.059406           0.049505         0
1   Aymeric Laporte         0.050746           0.029851         0
2    Raphael Varane         0.050132           0.018470         0
3   Clement Lenglet         0.046429           0.017857         0
4  Andrew Robertson         0.046196           0.086739         0

These are the algorithms:
1. K-Means Clustering
2. KNN Classification

Enter your choice:
1

You have chosen: K-Means Clustering

Enter the value of k: 3

Distances that can be used:
1. Manhattan Distance
2. Euclidean Distance

Choose the type of distance you want to use: 2
K-MEANS

Centroids:  [29, 3, 17]


Cluster  0 :

[10, 12, 13, 14, 15, 16, 17, 18, 19]
                 Names
10      Diego Maradona
12   Cristiano Ronaldo
13  Robert Lewandowski
14      Erling Haaland
15      Kyllian Mbappe
16         Luis Suarez
17  Zlatan Ibrahimovic
18       Neymar Junior
1

---
